In [1]:
## Importing required libraries
import sqlite3
import csv
import pandas as pd
import numpy as np
import seaborn as sns
import itertools
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer
from time import time
from sklearn.decomposition import PCA, FastICA
from sklearn.pipeline import Pipeline
import warnings
import random
import time
import datetime

warnings.simplefilter("ignore")

In [45]:
print ("Reading data...")

data = []
labels = []
featureName = []
#file = open("washed_data.csv")
file = open("washed_data2.csv")
for l in csv.reader(file):
    data.append(l[1:-1])
    labels.append(l[-1])
featureName = data[0]
del data[0]
del labels[0]

print ("done")

Reading data...
done


In [4]:
featureName

['home_team_goals_difference',
 'away_team_goals_difference',
 'games_won_home_team',
 'games_won_away_team',
 'games_against_won',
 'games_against_lost',
 'match_time_difference',
 'last_matches_points_home',
 'last_matches_points_away',
 'League_1.0',
 'League_1729.0',
 'League_4769.0',
 'League_7809.0',
 'League_10257.0',
 'League_13274.0',
 'League_15722.0',
 'League_17642.0',
 'League_19694.0',
 'League_21518.0',
 'League_24558.0',
 'home_player_1_overall_rating',
 'home_player_2_overall_rating',
 'home_player_3_overall_rating',
 'home_player_4_overall_rating',
 'home_player_5_overall_rating',
 'home_player_6_overall_rating',
 'home_player_7_overall_rating',
 'home_player_8_overall_rating',
 'home_player_9_overall_rating',
 'home_player_10_overall_rating',
 'home_player_11_overall_rating',
 'away_player_1_overall_rating',
 'away_player_2_overall_rating',
 'away_player_3_overall_rating',
 'away_player_4_overall_rating',
 'away_player_5_overall_rating',
 'away_player_6_overall_ratin

In [46]:
### pre-process on X and y
dataWash = []
labelsNew = []
for i in range(len(data)):
    add = 1
    timeDif = float(data[i][6])
    data[i][6]=timeDif/3600/24
    
    for j in range(len(data[i])):
        data[i][j] = float(data[i][j])
    dataWash.append(data[i])
    labelsNew.append(labels[i])
data = dataWash
labels = labelsNew

In [47]:
### split train, validation, test data
trainX = []
trainy = []
validateX = []
validatey = []
testX = []
testy = []
for i in range (len(data)):
    rdm = np.random.random()
    if rdm<0.6:
        trainX.append(data[i])
        trainy.append(labels[i])
    elif rdm<0.8:
        validateX.append(data[i])
        validatey.append(labels[i])
    else:
        testX.append(data[i])
        testy.append(labels[i])

In [48]:
###  calculate accuracy
def getAcc(predicty, testy):
    correctCnt = 0
    for i in range(len(testy)):
        if predicty[i] == testy[i]:
            correctCnt += 1
    return correctCnt*1.0/len(testy)

def getAccByLeague(predicty, testy, testX):
    matchNumByLeague = [0]*11
    correctByLeague = [0]*11
    testMatchNum = len(predicty)
    for i in range(testMatchNum):
        label = testy[i]
        pred = predicty[i]
        for j in range(7,18,1):
            if int(testX[i][j])==1:
                matchNumByLeague[j-7]+=1
                if label==pred:
                    correctByLeague[j-7] += 1
    accuracyByLeague = []
    for i in range(11):
        if matchNumByLeague[i]==0:
            accuracyByLeague.append(0)
        else:
            accuracyByLeague.append(correctByLeague[i]*1.0/matchNumByLeague[i])
    return accuracyByLeague

In [49]:
###  baseline: predict every match as home team wins
predicty = ['Win']*len(testy)
acc = getAcc(predicty,testy)
print ("accuracy of baseline is "+str(acc))

accuracy of baseline is 0.45911096029860876


In [50]:
###   try logistic regression
logClf = linear_model.LogisticRegression(multi_class = "ovr", solver = "sag", class_weight = 'balanced')
logClf.fit(trainX,trainy)
predicty = logClf.predict(testX)
acc = getAcc(predicty,testy)
print ("accuracy of LR is "+str(acc))

accByLeague = getAccByLeague(predicty,testy,testX)
print(accByLeague)

accuracy of LR is 0.497115710892433
[0.4654427645788337, 0.49427168576104746, 0.49444444444444446, 0.491566265060241, 0.4953051643192488, 0.4550408719346049, 0.4804177545691906, 0.5290697674418605, 0, 0.5638297872340425, 0.47368421052631576]


In [51]:
###  only use lottery features
trainXNew = []
testXNew = []
for row in trainX:
    trainXNew.append(row[-6:])
for row in testX:
    testXNew.append(row[-6:])
logClf = linear_model.LogisticRegression(multi_class = "ovr", solver = "sag", class_weight = 'balanced')
logClf.fit(trainXNew,trainy)
predicty = logClf.predict(testXNew)
acc = getAcc(predicty,testy)
print ("accuracy of LR is "+str(acc))
accByLeague = getAccByLeague(predicty,testy,testX)
print(accByLeague)

accuracy of LR is 0.4872751951136749
[0.46220302375809935, 0.48226950354609927, 0.4666666666666667, 0.5108433734939759, 0.4507042253521127, 0.4550408719346049, 0.5117493472584856, 0.4883720930232558, 0, 0.5106382978723404, 0.47368421052631576]


In [52]:
###  try gaussian naive bayes
GNBClf = GaussianNB()
GNBClf.fit(trainX,trainy)
predicty = GNBClf.predict(testX)
acc = getAcc(predicty,testy)
print ("accuracy of GNB is "+str(acc))

accByLeague = getAccByLeague(predicty,testy,testX)
print(accByLeague)

accuracy of GNB is 0.4825246012894469
[0.4524838012958963, 0.4735406437534097, 0.4666666666666667, 0.5253012048192771, 0.4131455399061033, 0.4359673024523161, 0.46736292428198434, 0.4941860465116279, 0, 0.5106382978723404, 0.5307017543859649]


In [53]:
###  try random forest
RFClf = RandomForestClassifier(n_estimators = 200, random_state = 1, class_weight = 'balanced')
RFClf.fit(trainX,trainy)
predicty = RFClf.predict(testX)
acc = getAcc(predicty,testy)
print ("accuracy of RF is "+str(acc))

accByLeague = getAccByLeague(predicty,testy,testX)
print(accByLeague)

accuracy of RF is 0.5256192738378012
[0.5005399568034558, 0.5231860338243317, 0.49444444444444446, 0.5397590361445783, 0.48826291079812206, 0.47956403269754766, 0.5274151436031331, 0.5406976744186046, 0, 0.5638297872340425, 0.5307017543859649]


In [54]:
###  try adaboost
ABClf = AdaBoostClassifier(n_estimators = 200, random_state = 2)
ABClf.fit(trainX,trainy)
predicty = ABClf.predict(testX)
acc = getAcc(predicty,testy)
print ("accuracy of AB is "+str(acc))

accByLeague = getAccByLeague(predicty,testy,testX)
print(accByLeague)

accuracy of AB is 0.499151679674245
[0.46490280777537796, 0.4909983633387889, 0.45555555555555555, 0.4891566265060241, 0.44366197183098594, 0.46321525885558584, 0.5143603133159269, 0.5261627906976745, 0, 0.5797872340425532, 0.5087719298245614]


In [55]:
###  try KNN
KNNClf =  KNeighborsClassifier()
KNNClf.fit(trainX,trainy)
predicty = KNNClf.predict(testX)
acc = getAcc(predicty,testy)
print ("accuracy of KNN is "+str(acc))

accByLeague = getAccByLeague(predicty,testy,testX)
print(accByLeague)


accuracy of KNN is 0.44044791313199866
[0.4136069114470842, 0.4402618657937807, 0.4222222222222222, 0.46265060240963857, 0.4154929577464789, 0.3869209809264305, 0.4046997389033943, 0.4622093023255814, 0, 0.526595744680851, 0.4517543859649123]
